# DapticsClient - Introduction <a class="tocSkip">

This notebook contains an interactive introduction to the Python DapticsClient class,
a simplified interface for accessing the Daptics GraphQL API for the optimization of
experimental design.

Documentation for using the DapticsClient class (implemented in the daptics_client.py
file in this folder) is included as comment lines in the interactive Python cells of
this notebook.

For additional help or information, please visit or contact Daptics.

On the web at https://daptics.ai
By email at support@daptics.ai

Daptics API Version 0.12.0  
Copyright (c) 2020 Daptics Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), the rights to use, copy, modify, merge, publish, and/or distribute, copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

You do not have the right to sub-license or sell copies of the Software.

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Connect and start a session

## Step 1. Create a DapticsClient object and connect to the API server. <a class="tocSkip">

Before running this project, please make sure that your Jupyter Python environment supports Python 3, and has these required packages installed:

* chardet
* urllib3
* requests
* gql
* async_timeout
* websockets

You will also need a validated user account on the Daptics API server.  You can create an account by [registering](https:daptics.ai/register) at https://daptics.ai

See the 01_README.ipynb notebook in this folder for more information.

The following cell creates a DapticsClient object and connect to the API server.
The client object constructor takes two optional arguments: either a URL for the
server you wish to use (scheme, host and port only), or the path to a JSON configuration
file. If both arguments are omitted, configuration will be read from environment
variables.

For information on the format of the JSON configuration file, and the configuration
environment variables, see the documentation  in the daptics_client.py file.

In [ ]:
# Import classes from the daptics_client module
# Requirements are Python 3, and the `gql` and `requests` libraries
from daptics_client import DapticsClient, DapticsTaskType, DapticsExperimentsType
from datetime import datetime
from time import sleep

# Create a client object using the development test API server URL.
api_host = 'http://localhost:4040'
daptics = DapticsClient(api_host)

# Show the (default) runtime options used by the client object
print("Client options: ", daptics.options)

# The `connect` method will attempt to connect to the API server and
# obtain the GraphQL schema.
daptics.connect()

__Notes:__

1. After creating a DapticsClient object, you can set options that affect
how the client functions. If you use the default options, it is your responsibility
to request each design generation. If you would like the design to be generated
automatically each time you submit experiments, you can set the `auto_generate_next_design`
option to `True`.
2. The default options also make it your responsibility to poll or wait
on asynchronous operations. If you would like to have the client do the waiting
for you automatically, you can set the `auto_task_timeout` option to a positive number,
representing the maximum number of seconds to wait for.
3. After connecting to the API server, the `daptics` object should have a `gql` attribute.
You can look at the data stored in the `gql` attribute by printing `daptics.gql.__dict__`.
In this data, you can see that the gql library has introspected all the GraphQL type,
query and mutation information exposed by the API.

## Step 2. Log in to the API server to obtain an access token. <a class="tocSkip">

The `login` method takes two string arguments, the user's `email` and `password`.

__Notes:__

1. You can also specify login credentials in a JSON configuration file, or read them from
environment variables--see the documentation  in the daptics_client.py file.
2. Use the real email address and password you used when you created
your account on the daptics.ai website.

In [ ]:
email = 'YOUR_EMAIL@YOUR_DOMAIN'
password = 'YOUR_PASSWORD'
login_data = daptics.login(email, password)

## Step 3. Create a daptics session on the server. <a class="tocSkip">

The `create_session` method takes two required arguments:
* `name` - the name for the session (which must be unique for your account)
* `description` - a short description

The session will be created for the `userId` saved from the login method.

Note: *each session name must be a new, unique name*. So, every time the following cellis executed, the session name must be changed.

In [ ]:
name = datetime.now().strftime('Practice Session %Y%m%d-%H%M%S')
description = 'This is a practice session'
daptics.create_session(name, description);

__Note__: if you want to start a new session, you may execute the cell above, changing the name. It is not necessary to restart the Python kernel.

# Experimental Space Definition

## Step 4. Specify the _Experimental Space Definition_.  <a class="tocSkip">

The _Experimental Space Definition_ (ESD) may be defined by a
CSV file that defines the names and constraints for each parameter. There are currently two different types of experiments, `mixture` and `factorial` (see tutorial notebook `02_Terminology`):

For `mixture` experiments, each row has exactly four columns:
* the parameter name (a string)
* the parameter type, set to `unit` for mixture experiments
* the minimum number of units for the parameter (an integer)
* the maximum number of units for the parameter (an integer)

For `factorial` experiments, each row has at least four columns,
but can have many more:
* the parameter name (a string)
* the parameter type, set to either `numerical` or `categorical`
* the first possible value for the parameter (a number)
* the second possible value for the parameter (a different number)
* ...etc

Each variable may have up to 20 values specified.

You should add blank padding columns for rows that have
fewer parameter values, so that the CSV file has the same number of
columns on each row.

Your ESD file should be placed in the directory that
the Jupyter notebook server process was started. The current location of this tutorial is
`../daptics-api/python_client/`; the tutorial file should appear in the `HOME` tab of the Jupyter notebook.

### Please note: running this tutorial will be free of charge for any ESDs that fit in the Daptics [Free zone](https://daptics.ai/pdt_pricing).
For this tutorial, we will use a free ESD, specified in the `esd-factorial-4x8.csv` file
installed in `../daptics-api/python_client/`.
To read your own ESD file, you would substitute your file name
for `esd-factorial-4x8.csv`.
The contents of the ESD file read as follows:


In [ ]:
fname = 'esd-factorial-4x8.csv'
with open(fname, 'r') as f:
    print(f.read())

## Step 5. Define the experimental space parameters for the session. <a class="tocSkip">

The `put_experimental_parameters_csv` method requires two
arguments:
* `params` - a dict of parameters that specify
    * `'space'`: a dict to define the space type, either
        * `{'type': 'factorial'}` or
        * `{'type': 'mixture', 'totalunits' : N}`, where `N` is the total number of units that all the
        mixture variables must sum to.
    * `'populationSize'`: how many experiments will be designed for each generation, and
    * `'replicates'`: how many replicates of an experiment will be performed
    (Note: to perform each experiment in the design just once, `replicates` should be zero.)
* `fname` - The path to the _Experimental Space Definition_ file discussed above.

In [ ]:
# Here is an example for a "mixture" space, which must include a
# `totalUnits` number:
#params = {
#    'space': {
#        'type': 'mixture',
#        'totalUnits': 15
#    },
#    'populationSize': 30,
#    'replicates': 2
#}
#fname = 'esd-mixture-5.csv'


# OR:  here is an example for a "factorial" space, which doesn't need the
# `totalUnits` parameter:
params = {
    'space': {
        'type': 'factorial'
    },
    'populationSize': 30,
    'replicates': 2
}
fname = 'esd-factorial-4x8.csv'

# When we call this method, the session will validate our parameters
# and if the validation succeeds, a potentially long-running process
# to set up the exploration space will be started.
task = daptics.put_experimental_parameters_csv(fname, params)

## Step 6. Poll the task and process results. <a class="tocSkip">

Execution of the cell above launches an asynchronous task to validate the
_Experimental Space Definition_ and set up the space.

The first cell below will poll this task using the `poll_for_current_task` method.
This call may be repeated, ruturning `'status': 'running'` while the task
is still in progress.

When the task is done (`'status': 'success'`), the results are then processed,
and the task is removed from the queue. Subsequent calls to `poll_for_current_task`
will result in an error because no task will be found.

__Notes__:

1. When the task's status changes to `'success'` in the session on the server,
the results will not be collected or processed  *until `poll_for_current_task` is called*.
2. You can also use the convenience method `wait_for_current_task` as illustrated
in the second cell below. `wait_for_current_task` simply wraps the `poll_for_current_task`
loop in a function, waiting for the task to finish, and returns the results (or errors)
when the task's status changes, or raises a timeout error if a timeout value
is provided.

In [ ]:
# Poll the task. Repeat until task disappears, when `status` is `success` (or `failure`).
while True:
    task, errors = daptics.poll_for_current_task()
    if task['currentTask'] is not None:
        status = task['currentTask']['status']
        print('status = ', status)
        if status == 'success':
            print(task)
            break
        sleep(0.5)
    else:
        print("No current task found!")
        break

In [ ]:
# OR-----------------

# Run the polling for the task in a loop, optionally specifying a maximum number of
# seconds to wait for a result.
daptics.wait_for_current_task(timeout=20.0)

## Step 7. Print session, save validated space. <a class="tocSkip">

If the _Experimental Space Definition_ was successfully validated, we may see it by printing out the current session state:

In [ ]:
daptics.print()

The validated _Experimental Space Definition_ object may be retrieved with the following call:

In [ ]:
space = daptics.get_experimental_space()

# Show all the experimental space parameters.
print(space)

and it may be exported to a CSV file (whose contents should be the same as the _Experimental Space Definition_ CSV file):

In [ ]:
fname = 'validated_space.csv'

# The export_csv method can be used to create a CSV file from any "table" element.
daptics.export_csv(fname, space['table'])

# Show the contents of the exported file.
with open(fname, 'r') as f:
    print(f.read())

# Initial Experiments

## Step 8. Optionally specify initial experiments. <a class="tocSkip">

Daptics modeling may be initialized with any available data from initial experiments done previously.  The data file for initial experiments must have a certain form: one row of header, followed by one row for each experiment.
We illustrate two different ways to create initial experiments.

### Step 8a. Initialize with data read from a CSV file. <a class="tocSkip">

The CSV data file may be created with the appopriate
column headers by using the `export_initial_experiments_template_csv` method:

In [ ]:
fname = 'initial_experiments.csv'
columns = daptics.export_initial_experiments_template_csv(fname)

# Print out the column names (parameter names and `Response`).
print(columns)

# Let's verify the contents of the exported file. It should just
# have one line listing the parameter names as well as a final column
# named `Response`, separated by commas.
#
# param1,param2,param3,param4,Response

with open(fname, 'r') as f:
    print(f.read())

Now that the file `initial_experiments.csv` has been created in the Jupyter Notebook home directory
(i.e. the directory where the Jupyter Notebook server was launched), you can add experiment
rows to the file, one row for each experiment, containing a value for each of the
experimental variables (the specification of the experiment) followed by that experiment's response.

You may also construct the initial experiments file from scratch,
but **it must have the correct column names in the header row**, including the `Response`
column.

### Step 8b. Initialize with random experiments and responses. <a class="tocSkip">

As an alternative to creating a CSV file of initial experiments,
the following cell creates random experiments with random responses,
just as an example for processing initial experiments.

In [ ]:
space = daptics.get_experimental_space()
design = None
num_extras = 20

# The random_experiments_with_responses method creates
# experiments. We set the design argument to None, because we don't have
# a generated design yet, and set the num_extras argument to the number of
# random experiments we would like to have generated.
random_experiments = daptics.random_experiments_with_responses(space, design, num_extras)

# Now let's save these to a file using the `export_csv` utility method.
fname = 'initial_experiments.csv'
task = daptics.export_csv(fname, random_experiments)

# Show the contents of the file we just saved.
with open(fname, 'r') as f:
    print(f.read())

## Step 9. Submit Initial Experiments. <a class="tocSkip">

If you have created a file of initial experiments by one of the two alternative
methods (Step 8a or Step 8b), you must upload them to the server before
the first experimental design can be generated.

When submitting experiments, you must specify the type of experiments that
are being sent to the server, in this case `DapticsExperimentsType.INITIAL_EXTRAS_ONLY`.

If you do not have any initial experiments to include in the session, you
may skip to Step 10.

In [ ]:
fname = 'initial_experiments.csv'
daptics.put_experiments_csv(DapticsExperimentsType.INITIAL_EXTRAS_ONLY, fname)

## Step 10. Generate the first design. <a class="tocSkip">

__Note__: This step could have been automated by setting the `auto_generate_next_design`
option in the client object, in which case the previous call to `put_experiments_csv`
would automatically start the design generation.

In [ ]:
# Generate a design, whether or not initial experiments were submitted.
task = daptics.generate_design()

## Step 11. Poll the task, print results <a class="tocSkip">

As in Step 6, the following call will poll the task launched in the previous cell
to create the first design, and return when the results are available.

In [ ]:
# Run the polling for the task in a loop, optionally specifying a maximum number of
# seconds to wait for a result.
daptics.wait_for_current_task(timeout=20.0)

In the following cell, when you print the state of the session,
you should now see the Design printed after the Experimental Space Definition.  Note that the Design printed has no responses in the final column.

In [ ]:
daptics.print()

# Process the experimental design

## Step 11. Save the current design to a CSV file.  <a class="tocSkip">
Once the current generation design is created, save it to
a CSV file. The `export_generated_design_csv` method saves the last generated design.

In [ ]:
# Create a file name for current generation, e.g. 'gen1_design.csv'
fname = 'gen{}_design.csv'.format(daptics.gen)

# Export the design
design = daptics.export_generated_design_csv(fname)

# Show the contents of the exported file, which will contain
# the designed parameter values and a blank `Response` column
# that we will fill in after running the designed experiments.
print("Current design in file", fname)
with open(fname, 'r') as f:
    print(f.read())

# Show the design (in Python) as well.
print(design)

## Step 12. Specify experimental responses. <a class="tocSkip">

We illustrate two alternative ways to record the responses to the experiments that have
been designed for you. Choose either Step 12a or 12b below.

### Step 12a. Specify your real experimental responses. <a class="tocSkip">

The previous cell has written the designed experiments to a file named
`gen1_design.csv`.

First copy that file to a new file named `gen1_experiments.csv`.
Then, using a spreadsheet or a text editor,
fill in a numerical (floating point) response value for each experiment row,
in the last column of each row.

After saving this file, return to this notebook to
show the response file contents.

In [ ]:
fname = 'gen{}_experiments.csv'.format(daptics.gen)

# Print them out
print("Responses for current design in file", fname)
with open(fname, 'r') as f:
    print(f.read())

### Step 12b. Specify random responses. <a class="tocSkip">
Create random response values for the
design we just generated using the client's
`random_experiments_with_responses` method.

We need to have the experimental space and the recently
generated design for the call to `random_experiments_with_responses`.
We do not want any extra experiments, so we pass `0` for the
`num_extras` argument.

In [ ]:
space = daptics.get_experimental_space()
design = daptics.get_generated_design()
num_extras = 0

# This time we must specify the design that we will be gnerating random responses for.
random_experiments = daptics.random_experiments_with_responses(space, design, num_extras)

Now let's save these to a file using the `export_csv` utility method.

In [ ]:
# E.g. gen1_experiments.csv
fname = 'gen{}_experiments.csv'.format(daptics.gen)
task = daptics.export_csv(fname, random_experiments)

# Show the contents of the file we just saved.
print("Random experiments in file", fname)
with open(fname, 'r') as f:
    print(f.read())

## Step 13. Submit these responses and automatically generate the next design. <a class="tocSkip">

Now we will upload these experiments to the session. This time we will set the runtime options
`auto_generate_next_design` and `auto_task_timeout`, to save us a few steps.

* Setting `auto_generate_next_design` to `True` will cause the client to immediately request
the next generated design.

* Setting `auto_task_timeout` to a postitive number will then wait on the design generation task
to complete.

__Note__: We must specify `DapticsExperimentsType.DESIGNED_WITH_OPTIONAL_EXTRAS` when submitting
experiments for the current design.

In [ ]:
# Change the client's runtime options.
daptics.options['auto_generate_next_design'] = True
daptics.options['auto_task_timeout'] = 60.0

# E.g. fname = 'gen1_experiments.csv'
fname = 'gen{}_experiments.csv'.format(daptics.gen)

# With the options set, this call will do three things:
# 1. upload generation 1 responses
# 2. request a generation 2 design (async task)
# 3. wait until the task is complete
result = daptics.put_experiments_csv(DapticsExperimentsType.DESIGNED_WITH_OPTIONAL_EXTRAS, fname)
print(result)

From this point on, you can repeat steps 11 through
13 until you are satisfied with your experimental
campaign, or until all the possible experimental
parameter combinations have been exhausted.

# Fully automated loop
The following loop illustrates how the execution of Steps 1–13 can be
fully automated. This approach may be helpful, for example,
if you generate your responses by means of simulations.

Please note that Daptics client options `auto_generate_next_design` and `auto_task_timeout` (described in the following section) are set, for easy execution.

In [ ]:
from daptics_client import DapticsClient, DapticsExperimentsType
from datetime import datetime

###################################################################
# Construct client object (Step 1)
# Uncomment if it you didn't run the previous steps.
# api_host = 'http://localhost:4040'
daptics = DapticsClient(api_host)

# Set up "shortcut" options to save time
daptics.options['auto_generate_next_design'] = True  # generate next design automatically after put_experiments
daptics.options['auto_task_timeout'] = -1   # wait for as long as it takes

# Connect to API server (Step 1)
daptics.connect()

# Login (Step 2) 
# Uncomment if you didn't run the previous steps, and fill in correct information
# email = 'YOUR_EMAIL@YOUR_DOMAIN'
# password = 'YOUR_PASSWORD'
daptics.login(email, password);

# Start a session (Step 3)
# Important: Session name must be changed each time this cell is executed.
name = datetime.now().strftime('Loop Session %Y%m%d-%H%M%S')
description = 'This is a full loop session'
daptics.create_session(name, description)
print("New session created")

###################################################################
# Experimental space (Steps 4-7)
params = {
    'space': {
        'type': 'factorial'
    },
    'populationSize': 30,
    'replicates': 2
}
fname = 'esd-factorial-4x8.csv'
# Save and validate the space
# Note: Because `auto_task_timeout` is set, we don't need to poll for result
task = daptics.put_experimental_parameters_csv(fname, params)
print("Experimental space done")

###################################################################
# Initial experiments (Steps 8-10)
space = daptics.get_experimental_space()
design = None
num_extras = 20
random_experiments = daptics.random_experiments_with_responses(space, design, num_extras)
fname = 'initial_experiments.csv'
daptics.export_csv(fname, random_experiments)
task = daptics.put_experiments_csv(DapticsExperimentsType.INITIAL_EXTRAS_ONLY, fname)
# Note: Because `auto_generate_next_design` is set, we don't need to call generate_design
# Note: Because `auto_task_timeout` is set, we don't need to poll for result
print("Initial experiments saved and generation 1 design done.")

# Save design in a file named for current generation, e.g. 'gen1_design.csv'
fname = 'gen{}_design.csv'.format(daptics.gen)
print("Saving design to", fname)
design = daptics.export_generated_design_csv(fname)

#######################################################################
# Loop: (Steps 12-13, repeated for 5 generations)
for n in range(5):
    ###################################################################
    # Fill design with random responses
    space = daptics.get_experimental_space()
    design = daptics.get_generated_design()
    num_extras = 0
    # Normally, you would fill the design with your experimental responses here.
    # For this tutorial we execute the following line
    # providing random numbers as a surrogate for the experimental responses.
    random_experiments = daptics.random_experiments_with_responses(space, design, num_extras)

    # Save the random experiments to a file, e.g. gen1_experimnts_csv.
    fname = 'gen{}_experiments.csv'.format(daptics.gen)
    task = daptics.export_csv(fname, random_experiments)

    # Upload responses and automatically generate next design
    result = daptics.put_experiments_csv(DapticsExperimentsType.DESIGNED_WITH_OPTIONAL_EXTRAS, fname)
    print("Done with design for generation", daptics.gen)

    # Save design in a file named for current generation, e.g. 'gen1_design.csv'
    fname = 'gen{}_design.csv'.format(daptics.gen)
    print("Saving design to", fname)
    design = daptics.export_generated_design_csv(fname)
print("Loop completed")

# Client options


After initializing a `dapics` object, set client options with calls like `daptics.options['option'] = value`.  The possible options are:

**1. auto_export_path option**

default:  `daptics.options['auto_export_path'] = None`

If you set the `auto_export_path` client option to an exisiting directory in the file system,
the validated space table, any submitted experiments, and any generated designs will be
save automatically, without your having to use the `export_csv` or `export_generated_design_csv`
methods.

**2. auto_generate_next_design option**

default:  `daptics.options['auto_generate_next_design'] = False`

If true, execute `generate_design()` automatically after `put_experiments()` or `put_experiments_csv()`.

**3. auto_task_timeout option**

default `daptics.options['auto_task_timeout'] = None`

Possible values are:

* N>0: (integer) number of seconds to wait for task to finsish (blocking, printing polling messages)
* N<0: poll forever
* None: no polling, must call  `wait_for_current_task()` or `poll_for_current_task()` (in a loop) to ensure task is finished.


# Calls that generate tasks

1. `put_experimental_parameters()` (or its csv variant, `put_experimental_parameters_csv()`) creates a "space" task
2. `generate_design()` creates a "generate" task

These should be followed by `wait_for_current_task()` or `poll_for_current_task()`